<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeAutomatizado/blob/master/notebooks/2c_regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresión lineal

Curso: [Aprendizaje Automatizado](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado). Profesor: [Gibran Fuentes Pineda](https://turing.iimas.unam.mx/~gibranfp/). Ayudantes: [Bere](https://turing.iimas.unam.mx/~bereml/) y [Ricardo](https://turing.iimas.unam.mx/~ricardoml/) Montalvo Lezama.

---
---

En esta libreta veremos un ejemplo paso a paso del modelo de regresión lineal con dos atributos de entrada y una salida. Emplearemos un conjunto de calificaciones de alumnos posgrado que tomaron primero el curso de Aprendizaje de Automatizado y después el curso de Aprendizaje Profundo. El conjunto tiene dos atributos: la calificación obtenida en el curso de Aprendizaje de Automatizado y el número de horas de estudio para el examen del curso de Aprendizaje Profundo. Como salida se tiene la calificación obtenida en examen de Aprendizaje Profundo.

## 1. Preparación

In [1]:
# para correr con widgets en local instalar jupyter-matplotlib
# https://github.com/matplotlib/jupyter-matplotlib
# descomentar la siguiente linea y correr de nuevo toda la libreta
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

ModuleNotFoundError: No module named 'ipympl'

## 2. Carga de datos

Leamos con Pandas:

In [ ]:
URL = 'https://raw.githubusercontent.com/gibranfp/CursoAprendizajeAutomatizado/master/data/califs.csv'
df = pd.read_csv(URL)
df.head(5)

Conversión de numpy:

In [ ]:
x = df.iloc[:,:2].values
print(x.shape)
x[:5]

In [ ]:
y_true = df.iloc[:,2].values
print(y_true.shape)
y_true[:5]

Construcción de la matriz de diseño:

In [ ]:
ones = np.ones([x.shape[0], 1])
x = np.hstack([ones, x])
print(x.shape)
x[:5]

Guardamos número de atributos y ejemplos:

In [ ]:
n, d = x.shape
n, d

## 3. Exploración

Grafiquemos para tener una idea de la distribución de los datos:

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[:, 1], x[:, 2], y_true,
           color='tab:blue')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y')
plt.show()

## 4. Hipótesis

Recordemos que dado un conjunto de ejemplos con atributos $x_1, x_2$ y salida $y$, la hipótesis de regresión lineal considerando un plano está dada por:

$$ \hat{y} = \theta_0 + x_1 \theta_1 + x_2 \theta_2 $$

donde $\theta_0, \theta_1, \theta_2$ son parámetros del modelo y $\hat{y}$ la salida predicha. Empleando la matriz de diseño, podemos expresar la hipótesis en su forma vectorial para todo el conjunto:

$$ \mathbf{\hat{y}} = \mathbf{x} \mathbf{\theta}^T $$

Nuestro trabajo consiste en estimar (aprender) los parámetros $\theta_0, \theta_1, \theta_2$. Por el momento supongamos que un oráculo nos regalo los siguientes valores para los parámetros y hagamos inferencia:

In [ ]:
theta = np.array([1.7071569, 0.13335178, 0.41122846])

y_pred = []
for i in range(n):
    y_p = theta[0] + x[i, 1] * theta[1] + x[i, 2] * theta[2]
    y_pred.append(y_p)
y_pred = np.array(y_pred)
y_pred[:5]

Implementando la forma vectorial:

In [ ]:
y_pred = x @ theta.T
y_pred[:5]

Grafiquemos el plano correspondiente a los parámetros propuestos:

In [ ]:
x1 = np.linspace(x[:,1].min(), x[:,1].max(), 2)
x2 = np.linspace(x[:,2].min(), x[:,2].max(), 2)
x1, x2 = np.meshgrid(x1, x2)
y_pred = theta[0] + theta[1] * x1 + theta[2] * x2

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x1, x2, y_pred, rstride=1, cstride=1, alpha=0.5)
ax.plot(x[:, 1], x[:, 2], y_true, 'b.')
ax.set_xlabel('x1')
ax.set_ylabel('x2')
ax.set_zlabel('y')
plt.show()

## 5. Función de pérdida

La pérdida para este modelo es el error cuadrático medio y queda expresado de la siguiente manera:

$$ E(\mathbf{\theta}) = \frac{1}{2n}  \sum_{i=1}^{n}{(\hat{y}^{(i)} - y^{(i)})^2} $$

en su forma vectorial:

$$ E(\mathbf{\theta}) = \frac{1}{2n} (\mathbf{\hat{y}} - \mathbf{y})^T (\mathbf{\hat{y}} - \mathbf{y}) $$

Para los parámetros propuestos, la pérdida se puede implementar como:

In [ ]:
# recomputamos las predicciones de los datos
y_pred = x @ theta.T

loss = 0
for i in range(n):
    loss += (y_pred[i] - y_true[i])**2
loss /= 2 * n
loss

y en su forma vectorial:

In [ ]:
loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
loss

## 6. Descenso por gradiente

![graddes](https://ml-cheatsheet.readthedocs.io/en/latest/_images/gradient_descent_demystified.png)

Imagen tomada de https://ml-cheatsheet.readthedocs.io/en/latest/gradient_descent.html

El algoritmo del descenso por gradiente se basa en el gradiente de la pérdida respecto de los parámetros:

$$ \frac{\partial E(\theta)}{\partial \theta_j} = \frac{1}{n} \sum_{i=1}^{n}{(\hat{y}^{(i)} - y^{(i)})x^{(i)}_j} $$

en su forma vectorial:

$$ \Delta E(\mathbf{\theta}) = \frac{1}{n} \mathbf{x}^T (\mathbf{\hat{y}} - \mathbf{y}) $$

Para los parámetros propuestos, el cómputo gradiente se puede implementar como:

In [ ]:
grad = [0, 0, 0]
for j in range(d):
    for i in range(n):
        grad[j] += (y_pred[i] - y_true[i]) * x[i, j]
    grad[j] /= n
np.array(grad)

y en su forma vectorial:

In [ ]:
grad = (x.T @ (y_pred - y_true)) / n
grad

Con base en el gradiente, la regla de actualización del descenso por gradiente queda expresada como:

$$ \theta_d^{[t+1]} \leftarrow \theta_d^{[t]}  - \alpha \frac{\partial E(\theta^{[t]})}{\partial \mathbf{\theta}_d^{[t]}} $$

mientras que su forma vectorial:

$$ \mathbf{\theta}^{[t+1]} \leftarrow \mathbf{\theta}^{[t]}  - \alpha \Delta E(\mathbf{\theta}^{[t]}) $$

Para los parámetros propuestos y datos, el gradiente se puede implementar como:

In [ ]:
alpha = 0.001

theta_next = []
for t, g in zip(theta, grad):
    t_next = t - alpha * g
    theta_next.append(t_next)
theta = np.array(theta_next)
theta

y su forma vectorial:

In [ ]:
# reniniciamos los parámetros
theta = np.array([1.7071569, 0.13335178, 0.41122846])

theta = theta - alpha * grad
theta

## 7. Entrenamiento

Realicemos 3 pasos del descenso por gradiente estocástico con lotes de 2 de forma manual. Primero inicialicemos de nuevo nuestros parámetros y tasa de aprendizaje: 

In [ ]:
alpha = 0.001
np.random.seed(0)
theta = np.random.normal(0, 1, d)
theta

Computemos la pérdida total de los datos y mantengamos un historial:

In [ ]:
y_pred = x @ theta.T
loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
loss_hist = [loss]
loss_hist

### 7.1. Paso 1

A) Selección del lote:

In [ ]:
examples = [0, 24]
n_batch = len(examples)
x_batch = x[examples]
x_batch

In [ ]:
y_true_batch = y_true[examples]
y_true_batch

B) Predicciones:

$$ \mathbf{\hat{y}} = \mathbf{x} \mathbf{\theta}^T $$

In [ ]:
y_pred_batch = x_batch @ theta.T
y_pred_batch

C) Pérdida:
$$ E(\mathbf{\theta}) = \frac{1}{2n} (\mathbf{\hat{y}} - \mathbf{y})^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
loss_batch = (y_pred_batch - y_true_batch).T @ (y_pred_batch - y_true_batch) / (2 * n_batch)
loss_batch

D) Gradiente:

$$ \Delta E(\mathbf{\theta}) = \frac{1}{n} \mathbf{x}^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
grad_batch = (x_batch.T @ (y_pred_batch - y_true_batch)) / n_batch
grad_batch

E) Parámetros actualizados:

$$ \mathbf{\theta}^{[t+1]} \leftarrow \mathbf{\theta}^{[t]}  - \alpha \Delta E(\mathbf{\theta}^{[t]}) $$

In [ ]:
theta = theta - alpha * grad_batch
theta

F) Actualicemos el historial de pérdidas:

In [ ]:
y_pred = x @ theta.T
loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
loss_hist.append(loss)
loss_hist

### 7.2 Paso 2

A) Selección del lote:

In [ ]:
examples = [8, 32]
n_batch = len(examples)
x_batch = x[examples]
x_batch

In [ ]:
y_true_batch = y_true[examples]
y_true_batch

B) Predicciones:

$$ \mathbf{\hat{y}} = \mathbf{x} \mathbf{\theta}^T $$

In [ ]:
y_pred_batch = x_batch @ theta.T
y_pred_batch

C) Pérdida:

$$ E(\mathbf{\theta}) = \frac{1}{2n} (\mathbf{\hat{y}} - \mathbf{y})^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
loss_batch = (y_pred_batch - y_true_batch).T @ (y_pred_batch - y_true_batch) / (2 * n_batch)
loss_batch

D) Gradiente:

$$ \Delta E(\mathbf{\theta}) = \frac{1}{n} \mathbf{x}^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
grad_batch = (x_batch.T @ (y_pred_batch - y_true_batch)) / n_batch
grad_batch

E) Parámetros actualizados:

$$ \mathbf{\theta}^{[t+1]} \leftarrow \mathbf{\theta}^{[t]}  - \alpha \Delta E(\mathbf{\theta}^{[t]}) $$

In [ ]:
theta = theta - alpha * grad_batch
theta

F) Actualicemos el historial de pérdidas:

In [ ]:
y_pred = x @ theta.T
loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
loss_hist.append(loss)
loss_hist

### 7.3. Paso 3

A) Selección del lote:

In [ ]:
examples = [16, 40]
n_batch = len(examples)
x_batch = x[examples]
x_batch

In [ ]:
y_true_batch = y_true[examples]
y_true_batch

B) Predicciones:

$$ \mathbf{\hat{y}} = \mathbf{x} \mathbf{\theta}^T $$

In [ ]:
y_pred_batch = x_batch @ theta.T
y_pred_batch

C) Pérdida:

$$ E(\mathbf{\theta}) = \frac{1}{2n} (\mathbf{\hat{y}} - \mathbf{y})^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
loss_batch = (y_pred_batch - y_true_batch).T @ (y_pred_batch - y_true_batch) / (2 * n_batch)
loss_batch

D) Gradiente:

$$ \Delta E(\mathbf{\theta}) = \frac{1}{n} \mathbf{x}^T (\mathbf{\hat{y}} - \mathbf{y}) $$

In [ ]:
grad_batch = (x_batch.T @ (y_pred_batch - y_true_batch)) / n_batch
grad_batch

E) Parámetros actualizados:

$$ \mathbf{\theta}^{[t+1]} \leftarrow \mathbf{\theta}^{[t]}  - \alpha \Delta E(\mathbf{\theta}^{[t]}) $$

In [ ]:
theta = theta - alpha * grad_batch
theta

F) Actualicemos el historial de pérdidas:

In [ ]:
y_pred = x @ theta.T
loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
loss_hist.append(loss)
loss_hist

## 8. Implementación

Esta es un implementación sencilla del descenso por gradiente en su forma vectorial.

In [ ]:
def train(x, y_true, alpha=0.01, steps=50):
    # ejemplos, atributos
    n, d = x.shape
    # inicialización de parámetros
    theta = np.random.normal(0, 1, d)
    # hostorial de pérdidas
    loss_hist = []
    # ciclo de entrenamiento
    for _ in range(steps):
        # cómputo de la hipótesis
        y_pred = x @ theta.T
        # cómputo de la pérdida
        loss = (y_pred - y_true).T @ (y_pred - y_true) / (2 * n)
        # cómputo del gradiente
        grad = (x.T @ (y_pred - y_true)) / n
        # actualización de parámetros
        theta = theta - alpha * grad
        # hostorial de pérdida
        loss_hist.append(loss)
    return theta, loss_hist

Entrenemos nuestro modelo:

In [ ]:
np.random.seed(0)
theta, loss_hist = train(x, y_true)
theta

Grafiquemos la evolución de la pérdida:

In [ ]:
plt.figure()
plt.plot(range(len(loss_hist)), loss_hist, 'r')
plt.show()

## 9. Tarea moral

Modificar esta libreta para implementar el descenso por gradiente estocástico.

![xkcd-linreg](https://imgs.xkcd.com/comics/linear_regression.png)

https://xkcd.com/1725/